# Лабораторная работа 3. Pandas, метод ближайших соседей и решающие деревья.

In [4]:
import numpy as np
import pandas as pd

## Часть 1: Pandas

#### Ответьте на вопросы о данных по авиарейсам в США.

Данные: http://stat-computing.org/dataexpo/2009/2008.csv.bz2
(обратите внимание, что распаковывать этот файл не обязательно — функция `pandas.read_csv` умеет читать из архивов автоматически)

Описание: http://stat-computing.org/dataexpo/2009/the-data.html

1. Какая из причин отмены рейса (`CancellationCode`) была самой частой? (расшифровки кодов можно найти в описании данных)
2. Найдите среднее, минимальное и максимальное расстояние, пройденное самолетом.
3. Не выглядит ли подозрительным минимальное пройденное расстояние? В какие дни и на каких рейсах оно было? Какое расстояние было пройдено этими же рейсами в другие дни?
4. Из какого аэропорта было произведено больше всего вылетов? В каком городе он находится?
5. Найдите для каждого аэропорта среднее время полета (`AirTime`) по всем вылетевшим из него рейсам. Какой аэропорт имеет наибольшее значение этого показателя?
6. Найдите аэропорт, у которого наибольшая доля задержанных (`DepDelay > 0`) рейсов. Исключите при этом из рассмотрения аэропорты, из которых было отправлено меньше 1000 рейсов (используйте функцию `filter` после `groupby`).

In [5]:
data = pd.read_csv("2008.txt")
data.describe()

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,FlightNum,ActualElapsedTime,...,Distance,TaxiIn,TaxiOut,Cancelled,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
count,1048575.0,1.048575e+06,1.048575e+06,1.048575e+06,1.014612e+06,1.048575e+06,1.011963e+06,1.048575e+06,1.048575e+06,1.011963e+06,...,1.048575e+06,1.011963e+06,1.014612e+06,1.048575e+06,1.048575e+06,270096.000000,270096.000000,270096.000000,270096.000000,270096.000000
mean,2008.0,1.422297e+00,1.572409e+01,3.895182e+00,1.343447e+03,1.331361e+03,1.489051e+03,1.497376e+03,2.359956e+03,1.244470e+02,...,6.881744e+02,6.825616e+00,1.678146e+01,3.238967e-02,2.526286e-03,15.708015,3.254199,15.878591,0.083955,21.974676
std,0.0,4.939256e-01,8.728132e+00,1.938000e+00,4.768043e+02,4.625290e+02,5.027085e+02,4.791596e+02,1.989882e+03,6.770997e+01,...,5.361000e+02,4.890826e+00,1.116645e+01,1.770328e-01,5.019866e-02,39.385498,20.354545,31.174628,1.728502,39.753498
min,2008.0,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.500000e+01,...,2.400000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2008.0,1.000000e+00,8.000000e+00,2.000000e+00,9.350000e+02,9.300000e+02,1.114000e+03,1.119000e+03,6.690000e+02,7.600000e+01,...,3.130000e+02,4.000000e+00,1.000000e+01,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2008.0,1.000000e+00,1.600000e+01,4.000000e+00,1.334000e+03,1.325000e+03,1.518000e+03,1.520000e+03,1.660000e+03,1.070000e+02,...,5.430000e+02,6.000000e+00,1.400000e+01,0.000000e+00,0.000000e+00,0.000000,0.000000,4.000000,0.000000,0.000000
75%,2008.0,2.000000e+00,2.300000e+01,5.000000e+00,1.733000e+03,1.720000e+03,1.912000e+03,1.908000e+03,3.818000e+03,1.530000e+02,...,8.990000e+02,8.000000e+00,2.000000e+01,0.000000e+00,0.000000e+00,16.000000,0.000000,20.000000,0.000000,28.000000
max,2008.0,2.000000e+00,3.100000e+01,7.000000e+00,2.400000e+03,2.359000e+03,2.400000e+03,2.400000e+03,9.202000e+03,6.930000e+02,...,4.962000e+03,2.130000e+02,3.830000e+02,1.000000e+00,1.000000e+00,1455.000000,1049.000000,1357.000000,149.000000,1143.000000


In [6]:
'''Самая частая причина отмены рейсов'''
canc=data['CancellationCode'].value_counts()
print(canc)
'''B is largest'''

B    14597
A    11519
C     7845
D        2
Name: CancellationCode, dtype: int64


'B is largest'

In [7]:
'''Average,min,max distance'''
avr=data['Distance'].mean()
print(avr)
min=data['Distance'].min()
print(min)
max=data['Distance'].max()
print(max)

688.1743785613809
24
4962


In [8]:
'''Min distance information'''
minDist=data.loc[data['Distance']==min]
year=int(minDist['Year'].to_string(index=False))
month=int(minDist['Month'].to_string(index=False))
day=int(minDist['DayofMonth'].to_string(index=False))
flightnum=int(minDist['FlightNum'].to_string(index=False))
print("Year:",year)
print("Month:",month)
print("Day:",day)

number=data.loc[data['FlightNum']==flightnum]

distances=number['Distance'].to_string(index=False)
print("Расстояние которое было пройдено этими же рейсами в другие дни")
print(distances)

Year: 2008
Month: 1
Day: 2
Расстояние которое было пройдено этими же рейсами в другие дни
613
 24
613
613
613
613
613
613
613
613
613
613
613
613
613
613
613
613
613
613
613
613
613
613
613
613
613
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
979
979
979
979
979
979
408
408
408
408
408
408
408
408
408
408
408
408
408
408
408
408
408
408
408
408
408
408
408
408
408
613
613
613
613
613
613
613
613
613
613
613
613
613
613
613
613
613
613
613
613
613
613
613
613
284
284
284
284
284
284
284
284
284
284
284
284
284
408
408
408
408
408
408
408
408
408
408
408
408
408


In [9]:
'''Из какого аэропорта было произведено больше всего вылетов'''
data['Origin'].mode().to_string(index=False)

'ATL'

In [10]:
'''Найдите для каждого аэропорта среднее время полета (AirTime) по всем вылетевшим из него рейсам. Какой аэропорт имеет наибольшее значение этого показателя?'''
means=data.groupby('Origin').AirTime.mean()
print(means)
print("Largest:",means.nlargest(1))

Origin
ABE     93.364771
ABI     36.531111
ABQ     90.510244
ABY     34.964646
ACT     30.509044
ACV     54.780488
ACY    110.741379
ADK    144.111111
ADQ     41.875000
AEX     69.689744
AGS     39.360465
ALB    105.704607
ALO     36.111111
AMA     55.697302
ANC    144.357345
ASE     59.554828
ATL     93.787326
ATW     61.966616
AUS     95.830714
AVL     78.154008
AVP    101.883333
AZO     37.080925
BDL    119.942156
BET     55.865854
BFL     64.077437
BGM     69.412844
BGR    108.115385
BHM     85.062088
BIL     69.914956
BIS     67.721805
          ...    
SPI     48.483740
SPS     29.945170
SRQ    107.288889
STL     97.555886
STT    164.979021
STX    179.022727
SUN     47.413965
SUX     44.882353
SWF    137.717421
SYR     85.776954
TEX     93.413333
TLH     70.629545
TOL     57.490196
TPA    118.624913
TRI     49.141509
TUL     83.792765
TUP     42.900000
TUS     89.057823
TVC     47.438492
TWF     42.261614
TXK     47.678571
TYR     31.864286
TYS     86.481397
VLD     45.037975
VPS

In [ ]:
'''Найдите аэропорт, у которого наибольшая доля задержанных (DepDelay > 0) рейсов. Исключите при этом из рассмотрения аэропорты, из которых было отправлено меньше 1000 рейсов (используйте функцию filter после groupby).'''
delays = data.groupby("Origin")['DepDelay']
filtered = data[data.DepDelay > 0].groupby("Origin")['DepDelay']
(filtered.count() / data.count() * (data.count() >= 1000)).argmax()

## Часть 2: метрические методы и категориальные признаки

In [22]:
import numpy as np
import pandas as pd

Все дальнейшие эксперименты предлагается проводить на данных соревнования Amazon Employee Access Challenge: https://www.kaggle.com/c/amazon-employee-access-challenge

В данной задаче предлагается предсказать, будет ли одобрен запрос сотрудника на получение доступа к тому или иному ресурсу. Все признаки являются категориальными.

Для удобства данные можно загрузить по ссылке: https://www.dropbox.com/s/q6fbs1vvhd5kvek/amazon.csv

Сразу прочитаем данные и создадим разбиение на обучение и контроль:

In [23]:
data = pd.read_csv('amazon.csv')
data.head()

,ACTION,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
0,1,39353,85475,117961,118300,123472,117905,117906,290919,117908
1,1,17183,1540,117961,118343,123125,118536,118536,308574,118539
2,1,36724,14457,118219,118220,117884,117879,267952,19721,117880
3,1,36135,5396,117961,118343,119993,118321,240983,290919,118322
4,1,42680,5905,117929,117930,119569,119323,123932,19793,119325


In [24]:
data.shape

(32769, 10)

In [25]:
# доля положительных примеров
data.ACTION.mean()

0.9421099209618847

In [27]:
# число значений у признаков
for col_name in data.columns:
    print (col_name, len(data[col_name].unique()))

ACTION 2
RESOURCE 7518
MGR_ID 4243
ROLE_ROLLUP_1 128
ROLE_ROLLUP_2 177
ROLE_DEPTNAME 449
ROLE_TITLE 343
ROLE_FAMILY_DESC 2358
ROLE_FAMILY 67
ROLE_CODE 343


In [28]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.iloc[:, 1:], data.iloc[:, 0],
                                                    test_size=0.3, random_state=241)

#### 1. Реализуйте три функции расстояния на категориальных признаках, которые обсуждались на втором семинаре.

Проще всего будет определить метрики как [user-defined distance](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.DistanceMetric.html), после чего воспользоваться реализацией kNN из sklearn (в этом случае используйте функцию predict_proba). Можно реализовать метод k ближайших соседей и самостоятально — в этом случае учитите, что он должен возвращать оценку вероятности, то есть отношение объектов первого класса среди соседей к числу соседей).

Постарайтесь уделить особое внимание эффективности кода — при реализации метрик "в лоб" вы можете столкнуться с очень большим временем выполнения.

#### Подсчитайте для каждой из метрик качество на тестовой выборке `X_test` при числе соседей $k = 10$. Мера качества — AUC-ROC.

#### Какая функция расстояния оказалась лучшей?

In [56]:
from sklearn.metrics import roc_auc_score
from sklearn.neighbors import DistanceMetric
from sklearn.neighbors import KNeighborsClassifier


In [31]:
'''Manhattan dist'''
manh = KNeighborsClassifier(n_neighbors = 10, metric = 'manhattan')
manh.fit(X_train,y_train)
y_true, y_pred = y_test, manh.predict(X_test)
roc_auc_score(y_true, y_pred)

0.5239873993707233

In [32]:
'''Euclidean dist'''
euc = KNeighborsClassifier(n_neighbors = 10, metric = 'euclidean')
euc.fit(X_train,y_train)
y_true, y_pred = y_test, euc.predict(X_test)
roc_auc_score(y_true, y_pred)

0.5176954413023741

#### 2 (бонус). Подберите лучшее (на тестовой выборке) число соседей $k$ для каждой из функций расстояния. Какое наилучшее качество удалось получить?

Для подбора можно использовать любые средства из sklearn.

#### 3. Реализуйте счетчики (http://blogs.technet.com/b/machinelearning/archive/2015/02/17/big-learning-made-easy-with-counts.aspx), которые заменят категориальные признаки на вещественные.

А именно, каждый категориальный признак нужно заменить на три: 
1. Число `counts` объектов в обучающей выборке с таким же значением признака.
2. Число `successes` объектов первого класса ($y = 1$) в обучающей выборке с таким же значением признака.
3. Сглаженное отношение двух предыдущих величин: (`successes` + 1) / (`counts` + 2).

Поскольку признаки, содержащие информацию о целевой переменной, могут привести к переобучению, может оказаться полезным сделать *фолдинг*: разбить обучающую выборку на $n$ частей, и для $i$-й части считать `counts` и `successes` по всем остальным частям. Для тестовой выборки используются счетчики, посчитанные по всей обучающей выборке. Реализуйте и такой вариант. Можно использовать $n = 3$.

#### Посчитайте на тесте AUC-ROC метода $k$ ближайших соседей с евклидовой метрикой для выборки, где категориальные признаки заменены на счетчики. Сравните по AUC-ROC два варианта формирования выборки — с фолдингом и без. Не забудьте подобрать наилучшее число соседей $k$.

#### 4. Добавьте в исходную выборку парные признаки — то есть для каждой пары $(f_i, f_j)$, $i < j$ исходных категориальных признаков добавьте новый категориальный признак $f_{ij}$, значение которого является конкатенацией значений $f_i$ и $f_j$ (желательно через какой-нибудь специальный символ во избежание коллизий). Посчитайте счетчики для этой выборки, найдите качество метода $k$ ближайших соседей с наилучшим $k$ (с фолдингом и без).

## Часть 3: Решающие деревья и леса

#### 1. Возьмите из предыдущей части выборку с парными признаками, преобразованную с помощью счетчиков без фолдинга. Настройте решающее дерево, подобрав оптимальные значения параметров `max_depth` и `min_samples_leaf`. Какой наилучший AUC-ROC на контроле удалось получить?

#### 2. Настройте случайный лес, подобрав такое число деревьев `n_estimators`, при котором ошибка выходит на асимптоту. Какое качество на тестовой выборке он дает?

#### 3. Возьмите выборку с парными признаками, для которой счетчики посчитаны с фолдингом. Обучите на ней случайный лес, подобрав число деревьев. Какое качество на тестовой выборке он дает? Чем вы можете объяснить изменение результата по сравнению с предыдущим пунктом?

Здесь вы можете поделиться своими мыслями о задании.

А здесь вставьте смешную картинку.

А здесь посоветуйте преподавателям хороший фильм или сериал.